In [5]:
# -----------------------------------------------------------
# 1. Load and Explore Visit Data from i2b2-Mimicking CSV
# -----------------------------------------------------------
# 📌 This notebook demonstrates how to load and inspect clinical notes
# encoded in BinHex format from a CSV mimicking the i2b2 `visit_dimension` table.
# Each record includes:
#   - Encounter number (encounter_num)
#   - Patient ID (patient_num)
#   - Visit dates (start_date, end_date)
#   - Location codes (location_cd, location_path)
#   - Encoded clinical notes (visit_blob)

# 🔧 Load necessary library
import pandas as pd

# 📂 Path to the data file
csv_path = "datafiles/i2b2_encounter_table.csv"

# ✅ Load data into a DataFrame
df = pd.read_csv(csv_path)

# 🔍 Display the first 10 records to preview the data structure
df.head(10)


,encounter_num,patient_num,start_date,end_date,inout_cd,location_cd,location_path,visit_blob
0,475303,1000000001,01/16/2003,01/16/2003,O,ASTHMA_CLINIC,\Hospital\Clinic\Pulmonary\Asthma\\,0x2A2A566973697420496E666F726D6174696F6E3A2A2A...
1,479681,1000000001,03/29/2007,03/29/2007,O,ASTHMA_CLINIC,\Hospital\Clinic\Pulmonary\Asthma\\,0x2A2A566973697420496E666F726D6174696F6E3A2A2A...
2,480315,1000000001,09/20/2007,09/20/2007,O,ASTHMA_CLINIC,\Hospital\Clinic\Pulmonary\Asthma\\,0x2A2A566973697420496E666F726D6174696F6E3A2A2A...
3,480903,1000000001,03/04/2008,03/04/2008,O,ASTHMA_CLINIC,\Hospital\Clinic\Pulmonary\Asthma\\,0x5468697320697320612032332D796561722D6F6C6420...
4,481398,1000000001,08/11/2008,08/11/2008,O,ASTHMA_CLINIC,\Hospital\Clinic\Pulmonary\Asthma\\,0x566973697420496E666F726D6174696F6E3A20202020...
5,482655,1000000001,05/18/2009,05/18/2009,O,ASTHMA_CLINIC,\Hospital\Clinic\Pulmonary\Asthma\\,0x566973697420496E666F726D6174696F6E3A0A202020...
6,471658,1000000002,04/17/1998,04/17/1998,O,GEN_MED_OUTPATIENT,\Hospital\Outpatient\GeneralMedicine\\,0x2A2A566973697420496E666F726D6174696F6E3A2A2A...
7,472076,1000000002,01/04/1999,01/04/1999,O,GEN_MED_OUTPATIENT,\Hospital\Outpatient\GeneralMedicine\\,0x2A2A566973697420496E666F726D6174696F6E3A2A2A...
8,472473,1000000002,08/12/1999,08/12/1999,O,GEN_MED_OUTPATIENT,\Hospital\Outpatient\GeneralMedicine\\,0x2A2A566973697420496E666F726D6174696F6E3A2A2A...
9,472692,1000000002,12/01/1999,12/01/1999,O,GEN_MED_OUTPATIENT,\Hospital\Outpatient\GeneralMedicine\\,0x2A2A566973697420496E666F726D6174696F6E3A2A2A...


In [6]:
# -----------------------------------------------------------
# 2. Interactive Exploration and Decoding of Clinical Notes
# -----------------------------------------------------------
# 📌 This cell allows interactive selection of patients and encounters
# to decode and preview their clinical notes stored in BinHex format.

import binascii
from IPython.display import display, Markdown
import ipywidgets as widgets

# 🔍 Create a list of distinct patient numbers
distinct_patients = sorted(df['patient_num'].unique())

# Create dropdown widgets for patient and encounter selection
patient_dropdown = widgets.Dropdown(
    options=distinct_patients,
    description="Patient #:",
    layout=widgets.Layout(width='50%')
)

encounter_dropdown = widgets.Dropdown(
    options=[],
    description="Encounter #:",
    layout=widgets.Layout(width='50%')
)

# Function to update encounters based on selected patient
def update_encounters(*args):
    patient_encounters = df[df['patient_num'] == patient_dropdown.value]['encounter_num'].tolist()
    encounter_dropdown.options = patient_encounters

# Function to decode and preview the clinical note
def decode_and_preview(patient_num, encounter_num):
    selected_row = df[
        (df['patient_num'] == patient_num) &
        (df['encounter_num'] == encounter_num)
    ].iloc[0]

    hex_blob = selected_row["visit_blob"].replace("0x", "")
    decoded_note = binascii.unhexlify(hex_blob).decode("utf-8", errors="ignore")
    display(Markdown(f"### 📝 Clinical Note Preview (Patient: {patient_num}, Encounter: {encounter_num})\n"))

    display(Markdown(decoded_note))

# Link dropdowns for dynamic interaction
patient_dropdown.observe(update_encounters, 'value')

# Initial population of encounter dropdown
update_encounters()

# Interactive widgets
widgets.interact(
    decode_and_preview,
    patient_num=patient_dropdown,
    encounter_num=encounter_dropdown
)


interactive(children=(Dropdown(description='Patient #:', layout=Layout(width='50%'), options=(np.int64(1000000…

<function __main__.decode_and_preview(patient_num, encounter_num)>

In [11]:
# -----------------------------------------------------------
# 3. Load the Ollama Model
# -----------------------------------------------------------
# Load the local LLM using LangChain's Ollama wrapper.

from langchain_ollama import ChatOllama

# Define the model name (assumed to be already pulled)
model_name = "qwen2"

model = ChatOllama(model=model_name)
print(f"✅ Model '{model_name}' is ready.")


✅ Model 'qwen2' is ready.


## 4. Basic Prompt Interaction - Simple Clinical Query
<img src="https://github.com/v-mourajr/mongan_llm_workshop/blob/master/images/basic_prompt.png?raw=true" alt="Basic Prompt" width="800">

In [12]:
# -----------------------------------------------------------
# 4. Run a Simple Clinical Query with System + User Prompts
# -----------------------------------------------------------
# This tests the model by asking a simple clinical question using structured messages.

from langchain_core.messages import HumanMessage, SystemMessage


messages = [
    SystemMessage(content=(
        "You are a knowledgeable medical provider. "
        "Provide clear, evidence-based explanations about a medical conditions."
    )),
    HumanMessage(content="What is asthma? What are its common symptoms and treatments?")
]

# Run inference
response = model.invoke(messages)

# Display result
print("🧠 Model Response:\n")
display(Markdown(response.content))


🧠 Model Response:



Asthma is a long-term (chronic) condition characterized by inflammation in the airways of the lungs. This inflammation causes narrowing or constriction of these passages, leading to difficulty breathing. The symptoms can vary widely from person to person but typically include:

### Common Symptoms of Asthma:
1. **Shortness of Breath**: Often described as breathlessness, especially during physical activity.
2. **Coughing**: Especially at night and early in the morning or upon waking.
3. **Wheezing**: A whistling sound when exhaling due to narrowed airways.
4. **Chronic Sputum Production**: Persistent cough with phlegm or mucus, especially if there is a history of recurring chest infections.

### Causes:
Asthma can be triggered by various factors including allergens (like pollen, dust mites), cold weather, exercise, strong emotions, medications, and some chemicals. The exact cause varies from person to person.

### Diagnosis:
Diagnosis usually involves understanding the patient's medical history, conducting a physical examination, and performing lung function tests like spirometry which measures how well your lungs are working.

### Treatments for Asthma:
Asthma management is personalized based on severity, frequency of symptoms, level of daily activity, and response to treatment. Common treatments include:

1. **Lifestyle Adjustments**: Avoiding known triggers can help manage symptoms.
2. **Inhalers**:
   - **Short-Acting Beta Agonists (SABAs)**: These are used for quick relief during an asthma attack.
   - **Long-Acting Beta Agonists (LABAs)**: Used in combination with other medications to control asthma symptoms on a daily basis. They should not be used as the sole treatment and require use of an inhaler that delivers corticosteroids.
3. **Corticosteroids**: These are often prescribed as a maintenance therapy, typically via an inhaler or a nebulizer, to reduce inflammation in the airways.
4. **Long-Term Control Medications**: Apart from LABAs, medications like Methylxanthines and Leukotriene Modulators can be used for long-term control of asthma.
5. **Allergy Shots (Immunotherapy)**: Useful if your asthma is triggered by allergies.

### Managing Asthma:
- Regular check-ups with a healthcare provider to review treatment effectiveness and adjust medications as needed.
- Keeping an asthma action plan that outlines symptoms, triggers, treatments, and when to seek medical help.
- Learning techniques like using a peak flow meter to monitor lung function at home can be part of self-management.

### Conclusion:
Asthma management is crucial for people with this condition. It often requires a combination of medications, avoidance strategies, and ongoing healthcare monitoring. By working closely with your doctor, you can effectively manage asthma symptoms and enjoy an active lifestyle.

## 5. Using ChatPromptTemplate for Dynamic Queries
<img src="https://github.com/v-mourajr/mongan_llm_workshop/blob/master/images/prompt_template.png?raw=true" alt="Basic Prompt" width="800">

In [33]:
# -----------------------------------------------------------
# 5. Create a Reusable Prompt Template
# -----------------------------------------------------------
# Demonstrates how to dynamically query different conditions.

from langchain.prompts import ChatPromptTemplate


patient_type = "child"
disease = "asthma"

messages = [
    ("system",
     "You are a knowledgeable medical provider. Provide clear, evidence-based explanations suitable for a {patient_type}."),

    ("human",
     "What is {disease}? What are its common symptoms and treatments?")
]

prompt_template = ChatPromptTemplate.from_messages(messages)

# Fill in variables
prompt = prompt_template.invoke({"patient_type": patient_type, "disease": disease})
response = model.invoke(prompt)

# Output
print("💬 AI Response:\n")
display(Markdown(response.content))


💬 AI Response:



Asthma is a condition that affects the airways in your lungs. Imagine your lungs as a tunnel where air travels from your nose or mouth all the way down to where you breathe out. When you have asthma, these tunnels can get inflamed and swollen, making them narrower. This makes it hard for air to go through, which can cause breathing problems.

**Common Symptoms of Asthma:**

1. **Coughing:** You might cough a lot, especially at night or early in the morning.
2. **Breathing困难:** It feels like you're struggling to breathe, and sometimes your chest may feel tight.
3. **Wheezing:** When you exhale, you can hear a whistling sound because air is rushing through narrow passages.
4. **Shortness of breath:** You might feel out of breath after just a little bit of activity.
5. **Fever:** Sometimes, if the asthma gets really bad and affects your body in more ways than breathing problems, you could have a fever.

**Treatments for Asthma:**

1. **Medicines to open up airways:** These can be taken as inhalers that help widen the tunnels so air can flow easily. Think of it like using a key to unlock a door.
2. **Long-term control medicine:** This is usually taken daily, often as a pill or inhaled through an inhaler, to keep your lungs healthy and reduce inflammation. It's like using a spray to keep plants healthy.
3. **Avoiding triggers:** Knowing what makes your asthma worse (like dust mites, pet hair, or strong smells) and trying to avoid them can help prevent symptoms.
4. **Emergency plan:** Sometimes breathing gets really bad, and you need quick action. There's a plan that helps with this too, which includes knowing when to use your fast-acting inhaler.

It's important to work closely with a doctor who specializes in asthma (an allergist or pediatrician) to find the best treatment for you. They can help adjust your medicine as needed and teach you how to manage your asthma effectively so you can live life without worrying too much about breathing difficulties.

In [34]:
# -----------------------------------------------------------
# 6. Clinical Note Extraction Prompt
# -----------------------------------------------------------
# This template prompts the LLM to extract structured information from a clinical note.

messages_notes = [
    ("system",
     "You are an advanced medical documentation assistant. Your task is to extract meaningful clinical insights from unstructured notes."),

    ("human",
     "Analyze the following clinical note: {patient_note}.\n\n"
     "Extract:\n"
     "1. Patient demographics\n"
     "2. Chief Complaints\n"
     "3. Current Medications\n"
     "4. Does the patient have asthma? (Yes/No)\n\n"
     "Respond in structured bullet points.")
]

prompt_template_notes = ChatPromptTemplate.from_messages(messages_notes)


In [36]:
# -----------------------------------------------------------
# 7. Analyze a Real Patient Note Using the Prompt
# -----------------------------------------------------------
# Pull a decoded note and run it through the clinical prompt

patient_num = 1000000047
encounter_num = 472602

selected_row = df[
    (df['patient_num'] == patient_num) &
    (df['encounter_num'] == encounter_num)
].iloc[0]

hex_blob = selected_row["visit_blob"].replace("0x", "")
decoded_note = binascii.unhexlify(hex_blob).decode("utf-8", errors="ignore")

# Fill the prompt with note text
filled_prompt = prompt_template_notes.invoke({"patient_note": decoded_note})

# Get model response
clinical_response = model.invoke(filled_prompt)

# Show results
print("📋 Extracted Clinical Information:\n")
display(Markdown(clinical_response.content))



📋 Extracted Clinical Information:



### Extracted Information

#### 1. Patient Demographics
- **Age:** 28 years
- **Ethnicity:** Black
- **Language:** English
- **Duration of care at clinic:** Approximately seven months
- **Date and time of appointment:** October 25, 1999

#### 2. Chief Complaints
- **Recurrent skin issues:** Facial and back acne exacerbation characterized by pain and persistence.
- **Symptoms affecting quality of life:** Painful and persistent acne, worsens during high stress or dietary changes.

#### 3. Current Medications
- **Oral antibiotics**
- **Topical retinoid**

#### 4. Asthma Status (Yes/No)
- **No**